## Decision Tree

1. Initialization: Start with the root node of the tree.

2. Splitting: At each node, determine the best split based on a chosen criterion (e.g., Gini impurity, information gain). This involves evaluating all possible splits for each feature and selecting the one that maximizes the criterion.

3. Stopping Criteria: Check stopping criteria to determine whether to stop growing the tree. Stopping criteria may include reaching the maximum depth, having fewer than a certain number of samples, or achieving perfect purity (homogeneity) in a leaf node.

4. Recursive Splitting: If the stopping criteria are not met, recursively split the data into subsets based on the chosen split criterion. Each subset becomes a child node of the current node.

5. Repeat: Repeat steps 2-4 for each child node until the stopping criteria are met for all nodes, or until no further improvement can be made.

6. Leaf Node Assignment: Once the tree has been built, assign a class label to each leaf node based on the majority class of the samples in that node.

7. Pruning (Optional): Prune the tree to prevent overfitting. Pruning techniques may involve removing nodes that do not significantly improve the performance of the tree on a validation set or applying post-pruning techniques to simplify the tree structure.

8. Final Decision Tree: The resulting tree represents the decision boundaries learned from the training data, with each internal node representing a decision based on a feature and each leaf node representing a class label.





Here's a more detailed breakdown of the steps involved in splitting a node (Step 2):

* Calculate impurity: Calculate the impurity measure for the current node based


* on the chosen criterion (e.g., Gini impurity, information gain). Impurity measures how mixed the classes are within the node.

* Evaluate potential splits: For each feature, evaluate potential split points to determine the best split that minimizes impurity. This involves sorting the feature values and evaluating the impurity at each split point.

* Select best split: Choose the feature and split point that result in the lowest impurity (or highest information gain) among all features and split points evaluated.

* Split the node: Split the node into two child nodes based on the chosen feature and split point. Samples with feature values below the split point go to the left child node, while samples with feature values above the split point go to the right child node.

* Repeat for child nodes: Recursively repeat the splitting process for each child node until the stopping criteria are met or no further improvement can be made.

This process continues recursively until all stopping criteria are met, resulting in the construction of the decision tree.








In [ ]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of the feature to split on
        self.threshold = threshold          # Threshold value for the feature
        self.left = left                    # Left child node
        self.right = right                  # Right child node
        self.value = value                  # Value (class label) if the node is a leaf

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth        # Maximum depth of the tree
        self.root = None                  # Root node of the decision tree

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_classes = len(set(y))

        # Stopping criteria
        if depth == self.max_depth or n_classes == 1 or n_samples < 2:
            return Node(value=self._most_common_label(y))

        # Find the best split
        best_feature, best_threshold = self._find_best_split(X, y)

        # Split the data
        left_indices = X[:, best_feature] < best_threshold
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[~left_indices], y[~left_indices]

        # Recursively build left and right subtrees
        left = self._build_tree(X_left, y_left, depth + 1)
        right = self._build_tree(X_right, y_right, depth + 1)

        return Node(feature_index=best_feature, threshold=best_threshold, left=left, right=right)

    def _find_best_split(self, X, y):
        best_gini = float('inf')
        best_feature, best_threshold = None, None

        for feature_index in range(X.shape[1]):
            thresholds = sorted(set(X[:, feature_index]))

            for threshold in thresholds:
                left_indices = X[:, feature_index] < threshold
                y_left = y[left_indices]
                y_right = y[~left_indices]

                gini = self._gini_impurity(y_left) + self._gini_impurity(y_right)

                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini_impurity(self, y):
        classes = set(y)
        impurity = 1.0

        for cls in classes:
            p_cls = sum(y == cls) / len(y)
            impurity -= p_cls ** 2

        return impurity

    def _most_common_label(self, y):
        return max(set(y), key=y.tolist().count)

    def predict(self, X):
        return [self._predict_sample(x, self.root) for x in X]

    def _predict_sample(self, x, node):
        if node.value is not None:
            return node.value
        elif x[node.feature_index] < node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the decision tree model from scikit-learn
sklearn_tree = DecisionTreeClassifier(random_state=42)
sklearn_tree.fit(X_train, y_train)

# Make predictions on the test set
sklearn_predictions = sklearn_tree.predict(X_test)

# Calculate accuracy of scikit-learn decision tree model
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)

# Fit the decision tree model implemented from scratch
scratch_tree = DecisionTreeClassifier(max_depth=3)
scratch_tree.fit(X_train, y_train)

# Make predictions on the test set using the scratch implementation
scratch_predictions = scratch_tree.predict(X_test)

# Calculate accuracy of scratch decision tree model
scratch_accuracy = accuracy_score(y_test, scratch_predictions)

# Print the accuracies
print("Accuracy of scikit-learn Decision Tree:", sklearn_accuracy)
print("Accuracy of Scratch Decision Tree:", scratch_accuracy)


Accuracy of scikit-learn Decision Tree: 1.0
Accuracy of Scratch Decision Tree: 1.0
